# Trainer Pattern

https://keras.io/examples/keras_recipes/trainer_pattern/

## Introduction
This example shows how to create a custom training step using the "Trainer pattern", which can then be shared across multiple Keras models. This pattern overrides the train_step() method of the keras.Model class, allowing for training loops beyond plain supervised learning.

The Trainer pattern can also easily be adapted to more complex models with larger custom training steps, such as this end-to-end GAN model, by putting the custom training step in the Trainer class definition.

## Data: 
https://www.kaggle.com/datasets/blastchar/telco-customer-churn


## Setup and Data Preparation

We begin by loading and preprocessing the **Telco Customer Churn** dataset, which contains structured information about telecom customers, such as service subscriptions, contract types, monthly charges, and whether the customer has left the service (churned).

This dataset allows us to showcase the **Trainer Pattern** for deep learning — a best practice for organizing model training into reusable, scalable components.

The trainer pattern is especially valuable in business environments where models are:
- Trained repeatedly across customer segments, product lines, or geographies  
- Updated frequently with new churn behavior or customer engagement data  
- Deployed in production systems that demand consistency, modularity, and rapid retraining  

By using churn prediction — a common and impactful business problem — we demonstrate how the trainer pattern supports modern **customer relationship management (CRM)** workflows and empowers data-driven retention strategies.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
import keras

# Load Telco Churn data
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Drop customer ID (not a useful feature)
df = df.drop("customerID", axis=1)

# Convert TotalCharges to numeric (some missing values are empty strings)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df = df.dropna()  # Drop rows with missing values

# Encode target
df["Churn"] = LabelEncoder().fit_transform(df["Churn"])  # 1 = churn, 0 = no churn

# One-hot encode categorical features
df = pd.get_dummies(df)

# Split into features and target
X = df.drop("Churn", axis=1).values
y = df["Churn"].values

# Scale numeric features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


## Define the Trainer Class

To make our deep learning workflow **modular, maintainable, and reusable**, we define a custom `MyTrainer` class by subclassing `keras.Model`.

This approach encapsulates the **training and evaluation steps** and allows any compatible model to use the same training logic — a key benefit when comparing multiple architectures or deploying at scale.

### What This Class Does:
- Uses **binary crossentropy** as the loss function for churn prediction (0 = stay, 1 = churn).
- Tracks **accuracy, precision, and recall** — critical metrics for understanding model performance in business contexts (e.g., how well we identify likely churners).
- Implements `train_step()` and `test_step()` methods to control how each batch is processed.
- Supports **gradient computation, backpropagation, and weight updates** manually via TensorFlow’s `GradientTape`.

This design mirrors how production machine learning systems are often built: with clearly defined components that can be audited, reused, and tested independently.


In [4]:
# Define a reusable training class using the Keras Model subclassing API
class MyTrainer(keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model  # Wrap the user-defined model

        # Define the loss function: BinaryCrossentropy for binary classification problems like churn prediction
        self.loss_fn = keras.losses.BinaryCrossentropy()

        # Define evaluation metrics
        self.accuracy_metric = keras.metrics.BinaryAccuracy()
        self.precision_metric = keras.metrics.Precision()
        self.recall_metric = keras.metrics.Recall()

    @property
    def metrics(self):
        # Return a list of metrics to be reset and tracked after each epoch
        return [self.accuracy_metric, self.precision_metric, self.recall_metric]

    def train_step(self, data):
        # Unpack the training data (features and labels)
        x, y = data

        # Record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # Forward pass: get model predictions
            y_pred = self.model(x, training=True)

            # Compute the loss between predictions and true labels
            loss = self.loss_fn(y, y_pred)

        # Compute gradients of the loss w.r.t. model weights
        trainable_vars = self.model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Apply gradients to update the model weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update all tracked metrics (e.g., accuracy, precision, recall)
        for metric in self.metrics:
            metric.update_state(y, y_pred)

        # Return a dictionary of current metric values
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack validation/test data
        x, y = data

        # Run inference (no training)
        y_pred = self.model(x, training=False)

        # Update metrics with validation predictions
        for metric in self.metrics:
            metric.update_state(y, y_pred)

        # Return a dictionary of current validation metric values
        return {m.name: m.result() for m in self.metrics}

    def call(self, x):
        # Forward pass when calling the model (used for inference or exporting)
        return self.model(x)


Tracking precision and recall gives more actionable insight than just accuracy, especially for imbalanced data. For example, high recall means your churn prevention team is catching more at-risk customers, even if a few false positives are included.

## Define Two Deep Learning Models for Churn Prediction

We define two neural network architectures to predict customer churn:

- `model_a` uses the **Sequential API**, which is simple and intuitive for stacking layers linearly.
- `model_b` uses the **Functional API**, which offers more flexibility and is preferred in production when models need branching or multiple inputs.

Both models:
- Take the Telco churn feature set as input.
- Use **dense (fully connected) layers** with ReLU activation to learn patterns.
- Include **Dropout** layers to reduce overfitting.
- End with a **sigmoid output** layer to predict the probability of churn (binary classification).

By keeping the architecture modular, we can easily reuse these models with our custom `Trainer` class and compare training behavior or performance across architectures — a common practice in applied business analytics workflows.


In [5]:
# Set input shape from your Telco data
input_shape = X_train.shape[1:]  # Example: (30,)

# Sequential API version
model_a = keras.models.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")  # Binary output
])

# Functional API version
inputs = keras.Input(shape=input_shape)
x = keras.layers.Dense(128, activation="relu")(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model_b = keras.Model(inputs, outputs)

These models are designed to predict whether a customer will churn, using features like contract type, tenure, and monthly charges. This setup can be reused across business units or time periods using the Trainer pattern

## Create Trainer Class Instances

Now that we've defined two different models (using the Sequential and Functional APIs), we can wrap them in our custom `MyTrainer` class.

This allows both models to share the same:
- Training loop
- Evaluation logic
- Loss function and metrics

This pattern ensures **consistent training behavior** across different models, which is especially useful when:
- Comparing architectures
- Running A/B tests in business applications
- Updating model logic while keeping evaluation consistent


In [6]:
trainer_1 = MyTrainer(model_a)
trainer_2 = MyTrainer(model_b)


## Compile and Train the Models

We now compile and train each model using different optimizers:

- `trainer_1`: uses SGD (Stochastic Gradient Descent), a classic optimizer useful for comparison.
- `trainer_2`: uses Adam, which is more adaptive and generally faster to converge for most deep learning tasks.

Both models share the same custom training and evaluation logic from the `MyTrainer` class, ensuring consistency across runs.


In [10]:
# Compile the first model with SGD optimizer
trainer_1.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01))
trainer_1.fit(
    X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test)
)

# Compile the second model with Adam optimizer
trainer_2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
trainer_2.fit(
    X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test)
)


Epoch 1/10
Epoch 1/10


 1/88 ━━━━━━━━━━━━━━━━━━━━ 28s 325ms/step - binary_accuracy: 0.7812 - precision: 0.5714 - recall: 0.5000

56/88 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - binary_accuracy: 0.8065 - precision: 0.6441 - recall: 0.5453 

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8047 - precision: 0.6465 - recall: 0.5439 - val_binary_accuracy: 0.7882 - val_precision: 0.6061 - val_recall: 0.5802
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8047 - precision: 0.6465 - recall: 0.5439 - val_binary_accuracy: 0.7882 - val_precision: 0.6061 - val_recall: 0.5802


Epoch 2/10
Epoch 2/10


...

88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8394 - precision_1: 0.7392 - recall_1: 0.6074 - val_binary_accuracy: 0.7918 - val_precision_1: 0.6392 - val_recall_1: 0.4973
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8394 - precision_1: 0.7392 - recall_1: 0.6074 - val_binary_accuracy: 0.7918 - val_precision_1: 0.6392 - val_recall_1: 0.4973


Epoch 10/10
Epoch 10/10


 1/88 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - binary_accuracy: 0.7656 - precision_1: 0.5882 - recall_1: 0.5556

56/88 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - binary_accuracy: 0.8149 - precision_1: 0.6726 - recall_1: 0.5533

88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8186 - precision_1: 0.6850 - recall_1: 0.5610 - val_binary_accuracy: 0.7846 - val_precision_1: 0.6092 - val_recall_1: 0.5294
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8186 - precision_1: 0.6850 - recall_1: 0.5610 - val_binary_accuracy: 0.7846 - val_precision_1: 0.6092 - val_recall_1: 0.5294


## Final Evaluation and Business Insights

In this notebook, we demonstrated how to apply deep learning to a real-world business problem — **customer churn prediction** — using a clean, reusable architecture known as the **Trainer Pattern**.

By structuring our model pipeline this way, we achieved:

- **Modularity**: We trained two different models (Sequential and Functional) with the same training logic, making experimentation fast and reliable.
- **Consistency**: Metrics like accuracy, precision, and recall were tracked uniformly, allowing us to fairly compare performance.
- **Scalability**: The `MyTrainer` class can be easily reused across departments or products, enabling rapid retraining as new customer data becomes available.

### Business Takeaways:
- Identifying customers likely to churn enables **proactive retention strategies** (e.g., special offers, outreach).
- A model with strong **recall** ensures you don’t miss many at-risk customers.
- A model with high **precision** helps ensure that retention efforts are well-targeted, avoiding wasted resources.

This project shows how deep learning, when implemented with scalable design patterns like the Trainer Pattern, becomes a practical and valuable tool for modern **business analytics** — not just for research or experimentation.

> With this reusable deep learning workflow, organizations can move from one-off churn models to **repeatable, maintainable, and strategic AI deployment.**
